This notebook is a basic tutorial that demonstrates how to configure a simulation using Concordia.

<a href="https://colab.research.google.com/github/google-deepmind/concordia/blob/main/examples/marketplace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Colab-specific setup (use a CodeSpace to avoid the need for this).
try:
  %env COLAB_RELEASE_TAG
except:
  pass  # Not running in colab.
else:
  %pip install --ignore-requires-python --requirement 'https://raw.githubusercontent.com/google-deepmind/concordia/main/examples/requirements.in' 'git+https://github.com/google-deepmind/concordia.git#egg=gdm-concordia'
  %pip list

In [ ]:
# @title Imports

from collections.abc import Sequence
import random
from typing import Any, Dict, List

from concordia.components import agent as actor_components
from concordia.components import game_master as gm_components
from concordia.contrib import language_models as language_model_utils
from concordia.contrib.components.game_master import marketplace
from concordia.environment.engines import simultaneous
import concordia.prefabs.entity as entity_prefabs
import concordia.prefabs.game_master as game_master_prefabs
from concordia.prefabs.simulation import generic as simulation
from concordia.typing import prefab as prefab_lib
from concordia.utils import helper_functions
from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sentence_transformers

In [ ]:
# @title Language Model Selection: provide key or select DISABLE_LANGUAGE_MODEL

# By default this colab uses models via an external API so you must provide an
# API key. TogetherAI offers open weights models from all sources.

API_KEY = ''  # @param {type: 'string'}
# See concordia/language_model/utils.py
API_TYPE = 'openai'  # e.g. 'together_ai' or 'openai'.
MODEL_NAME = (  # for API_TYPE = 'together_ai', we recommend MODEL_NAME = 'google/gemma-3-27b-it'
    'gpt-5'
)
# To debug without spending money on API calls, set DISABLE_LANGUAGE_MODEL=True
DISABLE_LANGUAGE_MODEL = False

In [ ]:
# @title Use the selected language model

# Note that it is also possible to use local models or other API models,
# simply replace this cell with the correct initialization for the model
# you want to use.

if not DISABLE_LANGUAGE_MODEL and not API_KEY:
  raise ValueError('API_KEY is required.')

model = language_model_utils.language_model_setup(
    api_type=API_TYPE,
    model_name=MODEL_NAME,
    api_key=API_KEY,
    disable_language_model=DISABLE_LANGUAGE_MODEL,
)

In [ ]:
# @title Setup sentence encoder

if DISABLE_LANGUAGE_MODEL:
  embedder = lambda _: np.ones(3)
else:
  st_model = sentence_transformers.SentenceTransformer(
      'sentence-transformers/all-mpnet-base-v2'
  )
  embedder = lambda x: st_model.encode(x, show_progress_bar=False)

In [ ]:
test = model.sample_text(
    'Is societal and technological progress like getting a clearer picture of '
    'something true and deep?'
)
print(test)

In [ ]:
# @title Load prefabs from packages to make the specific palette to use here.

prefabs = {
    **helper_functions.get_package_classes(entity_prefabs),
    **helper_functions.get_package_classes(game_master_prefabs),
}

In [ ]:
# @title Print menu of prefabs

display.display(
    display.Markdown(helper_functions.print_pretty_prefabs(prefabs))
)

In [ ]:
# @title Domain objects for marketplace simulations

Good = marketplace.Good
MarketplaceAgent = marketplace.MarketplaceAgent
MarketPlace = marketplace.MarketPlace


def make_goods() -> List[Good]:
  categories = ["Food", "Clothing", "Accessories", "Gadgets"]
  qualities = ["Low", "Mid", "High"]
  return [Good(c, q, c + "_" + q) for c in categories for q in qualities]


def make_agents(
    n: int, goods: Sequence[Good], names: Sequence[str], seed: int = 123
) -> List[MarketplaceAgent]:
  rng = random.Random(seed)
  agents: List[MarketplaceAgent] = []
  for i in range(n):
    role = "producer" if i < n // 2 else "consumer"
    inv_good = rng.choice(goods)
    inventory = {inv_good.id: rng.randint(5, 15)} if role == "producer" else {}
    agents.append(
        MarketplaceAgent(
            name=names[i],
            role=role,
            cash=50.0,
            inventory=inventory,
            queue=[],
        )
    )
  return agents

# 4 Actor Simulation

In [ ]:
goods = make_goods()
names = ['Alex', 'Nicole', 'Jeremy', 'Megan']
agents = make_agents(4, goods, names)

component_kwargs = {
    'components': [
        actor_components.observation.DEFAULT_OBSERVATION_COMPONENT_KEY
    ],
    'agents': agents,
    'goods': goods,
}

prefabs = {
    **helper_functions.get_package_classes(entity_prefabs),
    **helper_functions.get_package_classes(game_master_prefabs),
}

PLAYER_ONE = names[0]
PLAYER_TWO = names[1]
PLAYER_THREE = names[2]
PLAYER_FOUR = names[3]

instances = [
    prefab_lib.InstanceConfig(
        prefab='basic__Entity',
        role=prefab_lib.Role.ENTITY,
        params={
            'name': PLAYER_ONE,
            'goal': (
                'Your goal is to sell your stock of Food_Low for a profit. Your'
                ' cost to produce each unit is $2.00. You must try to sell for'
                ' more than $2.00 to be profitable. You will not accept a price'
                ' lower than your cost.'
            ),
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='basic__Entity',
        role=prefab_lib.Role.ENTITY,
        params={
            'name': PLAYER_TWO,
            'goal': (
                'Your goal is to sell your stock of Food_Low for a profit. You'
                ' are a very efficient producer, so your cost for each unit is'
                ' only $1.50. You must sell for more than $1.50, but you can'
                ' afford to undercut less efficient sellers.'
            ),
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='basic__Entity',
        role=prefab_lib.Role.ENTITY,
        params={
            'name': PLAYER_THREE,
            'goal': (
                'Your goal is to acquire Food_Low for your own use. You value'
                ' it highly, so you are willing to pay up to $4.50 per unit,'
                ' but you will try to get it for as cheap as possible. You will'
                ' not bid higher than $4.50.'
            ),
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='basic__Entity',
        role=prefab_lib.Role.ENTITY,
        params={
            'name': PLAYER_FOUR,
            'goal': (
                'Your goal is to acquire Food_Low. It is useful, but not'
                ' essential. You are willing to pay up to $3.75 per unit, but'
                ' you would prefer to pay much less. You will not bid higher'
                ' than $3.75.'
            ),
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='marketplace__GameMaster',
        role=prefab_lib.Role.GAME_MASTER,
        params={
            'name': 'MarketplaceGM',
            'experiment_component_class': MarketPlace,
            'experiment_component_init_kwargs': component_kwargs,
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='formative_memories_initializer__GameMaster',
        role=prefab_lib.Role.INITIALIZER,
        params={
            'name': 'initial setup rules',
            'next_game_master_name': 'MarketplaceGM',
            'shared_memories': [
                (
                    'There is a small town of Smallville where'
                    f' {PLAYER_ONE} and {PLAYER_TWO} grew up.'
                ),
            ],
        },
    ),
]

default_premise = """You are in a marketplace that buys and sells goods
"""

config = prefab_lib.Config(
    default_premise=default_premise,
    default_max_steps=5,
    prefabs=prefabs,
    instances=instances,
)

engine = simultaneous.Simultaneous()

#  Initialize the simulation
simul_sim = simulation.Simulation(
    config=config, model=model, embedder=embedder, engine=engine
)

In [ ]:
# @title Run the simulation
results_log = simul_sim.play()

In [ ]:
# @title Display the log
display.HTML(results_log)

# More Realistic Market of 10 Actors

In [ ]:
# @title Define real goods

DETAILED_GOODS = {
    "Food": {
        "Low": {
            "Street Taco": {
                "price": 2.0,
                "inventory": 100,
                "advert": (
                    "Authentic flavor, unbeatable price! Perfectly seasoned,"
                    " grilled to order, and served on a warm corn tortilla. The"
                    " true taste of the street."
                ),
            },
            "Cup Noodles": {
                "price": 2.5,
                "inventory": 100,
                "advert": (
                    "Quick, easy, and delicious. Your perfect savory meal is"
                    " just three minutes away. Just add hot water and enjoy."
                ),
            },
        },
        "Mid": {
            "Gourmet Burger": {
                "price": 15.0,
                "inventory": 50,
                "advert": (
                    "A burger experience like no other. A juicy, thick-cut"
                    " patty made with premium, locally-sourced beef, served on"
                    " a toasted brioche bun."
                ),
            },
            "Sushi Platter": {
                "price": 22.0,
                "inventory": 40,
                "advert": (
                    "A fresh and vibrant selection of our finest rolls and"
                    " nigiri. A taste of Japan in every artfully prepared bite."
                ),
            },
        },
        "High": {
            "Michelin Star Meal": {
                "price": 75.0,
                "inventory": 10,
                "advert": (
                    "An unforgettable culinary journey. Allow our celebrated"
                    " chef to present a multi-course tasting menu that will"
                    " delight your senses."
                ),
            },
            "Omakase Experience": {
                "price": 90.0,
                "inventory": 8,
                "advert": (
                    "Trust the chef. An intimate, curated selection of the"
                    " finest seasonal fish and delicacies, flown in daily and"
                    " prepared before your eyes."
                ),
            },
        },
    },
    "Clothing": {
        "Low": {
            "Brown Fast Fashion T-Shirt": {
                "price": 5.0,
                "inventory": 100,
                "advert": (
                    "Trendy, affordable, and effortlessly cool. Update your"
                    " style for any season without breaking the bank."
                ),
            },
            "White Cotton T-Shirt": {
                "price": 6.0,
                "inventory": 100,
                "advert": (
                    "The essential classic for any wardrobe. Made from 100%"
                    " soft, breathable cotton for a perfect, comfortable fit"
                    " every time."
                ),
            },
        },
        "Mid": {
            "Levi's Jeans": {
                "price": 80.0,
                "inventory": 50,
                "advert": (
                    "Quality never goes out of style. The original, iconic blue"
                    " jean, crafted from durable denim for a timeless look and"
                    " feel."
                ),
            },
            "AllSaints Leather Jacket": {
                "price": 100.0,
                "inventory": 25,
                "advert": (
                    "Define your edge. Cut from supple, premium lambskin"
                    " leather with signature metal hardware. A timeless"
                    " investment piece."
                ),
            },
        },
        "High": {
            "Armani Suit": {
                "price": 1200.0,
                "inventory": 10,
                "advert": (
                    "Exude confidence and power. Impeccable Italian tailoring"
                    " and luxurious fabrics combine for a silhouette that"
                    " commands attention."
                ),
            },
            "Burberry Jacket": {
                "price": 2500.0,
                "inventory": 5,
                "advert": (
                    "Iconic British luxury. The ultimate statement in"
                    " outerwear, featuring the classic check and unparalleled"
                    " craftsmanship."
                ),
            },
        },
    },
    "Gadgets": {
        "Low": {
            "Basic Headphones": {
                "price": 15.0,
                "inventory": 100,
                "advert": (
                    "Your daily audio companion. Delivers crisp, clear sound"
                    " for your music, podcasts, and calls, wherever you go."
                ),
            },
            "USB Power Bank": {
                "price": 20.0,
                "inventory": 100,
                "advert": (
                    "Never run out of power again. This compact and reliable"
                    " power bank ensures your devices stay charged on the go."
                ),
            },
        },
        "Mid": {
            "Kindle E-book Reader": {
                "price": 200.0,
                "inventory": 40,
                "advert": (
                    "Carry your library in one hand. A glare-free display that"
                    " reads like real paper, even in direct sunlight. Read"
                    " anytime, anywhere."
                ),
            },
            "Apple AirPods Pro": {
                "price": 225.0,
                "inventory": 30,
                "advert": (
                    "Magic, remastered. Experience a new level of immersive"
                    " sound with active noise cancellation and intuitive touch"
                    " controls."
                ),
            },
        },
        "High": {
            "Macbook Laptop": {
                "price": 1800.0,
                "inventory": 15,
                "advert": (
                    "Power to the pro. Supercharged by the latest chip, this"
                    " laptop delivers game-changing performance for your"
                    " biggest ideas."
                ),
            },
            "Sony 4K TV": {
                "price": 2000.0,
                "inventory": 10,
                "advert": (
                    "Picture-perfect reality. Experience breathtaking color,"
                    " contrast, and clarity that brings entertainment to life"
                    " in stunning 4K."
                ),
            },
        },
    },
    "Accessories": {
        "Low": {
            "Scaly the Lizard Beanie Baby": {
                "price": 8.0,
                "inventory": 100,
                "advert": (
                    "A collectible friend for all ages! This rare, retired"
                    " Beanie Baby is a must-have for any nostalgic collector."
                    " Don't miss out!"
                ),
            },
            "Temu Watch": {
                "price": 10.0,
                "inventory": 100,
                "advert": (
                    "Stylish and surprisingly affordable. Get the look of a"
                    " high-end watch for a fraction of the price. Why pay more?"
                ),
            },
        },
        "Mid": {
            "Stanley Cup Quencher Tumbler": {
                "price": 45.0,
                "inventory": 50,
                "advert": (
                    "The hydration must-have you never knew you needed. Made"
                    " from recycled stainless steel with vacuum insulation to"
                    " keep drinks cold for hours. Features an advanced"
                    " FlowState lid, an easy-carry handle, and a car cup"
                    " holder-friendly base."
                ),
            },
            "Longchamp Le Pliage Bag": {
                "price": 100.0,
                "inventory": 40,
                "advert": (
                    "Effortless Parisian chic. This iconic, foldable bag is the"
                    " perfect, lightweight companion for work, travel, and"
                    " every day."
                ),
            },
        },
        "High": {
            "Chanel Handbag": {
                "price": 1800.0,
                "inventory": 8,
                "advert": (
                    "An icon of elegance. The ultimate accessory for the"
                    " discerning fashionista. Rectangular shape, braided chain"
                    " strap, brand monogram clasp, double flap design, double"
                    " compartment, slip pockets at interior, polished hardware."
                ),
            },
            "Rolex Watch": {
                "price": 2000.0,
                "inventory": 5,
                "advert": (
                    "A crown for every achievement. Meticulously crafted from"
                    " Oystersteel and precious metals, this is more than a"
                    " timepiece—it is a legacy."
                ),
            },
            "Pop Mart Labubu Monster Vinyl Plush Doll (Limited Edition)": {
                "price": 280.0,
                "inventory": 15,
                "advert": (
                    "Extremely rare collector's item! Designed by Kasing Lung,"
                    " this limited-edition Labubu is the must-have for any"
                    " serious art toy enthusiast."
                ),
            },
        },
    },
}

In [ ]:
def make_agents_from_data(
    agent_data_list: List[Dict[str, Any]],
) -> List[MarketplaceAgent]:
  """Creates a list of MarketplaceAgent objects from agent data dicts."""
  agents: List[MarketplaceAgent] = []
  for agent_info in agent_data_list:
    inventory = agent_info.get("inventory", {})
    role = agent_info["type"]
    cash = float(agent_info.get("cash", 100.0))

    if role == "producer":
      inventory = {agent_info["good_to_sell"]: agent_info["inventory"]}
    elif role == "consumer":
      # make sure there's one piece of clothing in the inventory
      assert inventory, "Inventory should not be empty for consumer."

    agents.append(
        MarketplaceAgent(
            name=agent_info["name"],
            role=role,
            cash=cash,
            inventory=inventory,
            queue=[],
        )
    )
  return agents


def get_all_goods_from_spec(
    spec: Dict[str, Dict[str, Dict[str, Dict[str, Any]]]],
) -> List[Good]:
  """Creates a flat list of Good objects from the detailed specification."""
  goods_list: List[Good] = []
  for category, qualities in spec.items():
    for quality, items in qualities.items():
      for item_name, details in items.items():
        # Use .get() for optional attributes to avoid errors if they are missing
        price = details.get("price")
        inventory = details.get("inventory")
        advert = details.get("advert")

        goods_list.append(
            Good(
                category=category,
                quality=quality,
                id=item_name,
                price=price,
                inventory=inventory,
                advert=advert,
            )
        )
  return goods_list


goods = get_all_goods_from_spec(DETAILED_GOODS)

In [ ]:
names = [
    'Alex',
    'Nicole',
    'Jeremy',
    'Megan',
    'David',
    'Samantha',
    'Chris',
    'Jessica',
    'Ryan',
    'Lauren',
    'Alice',
    'Bob',
    'Charlie',
    'Drake',
    'Eve',
    'Frank',
    'Grace',
    'Heidi',
    'Ivan',
    'Judy',
]

num_rounds = 10
num_items = 1
num_agents = 10
num_sellers = num_agents // 2
num_buyers = num_agents - num_sellers

# --- Programmatically Generate num_agent Agent Profiles ---
random.seed(42)  # For reproducible assignments

if num_agents > len(names):
  # Generate generic names if there are more agents than unique names
  print(
      f'Info: {num_agents} agents > {len(names)} names. Generating generic'
      ' names.'
  )
  seller_profiles = [{'name': f'Seller_{i+1}'} for i in range(num_sellers)]
  buyer_profiles = [{'name': f'Buyer_{i+1}'} for i in range(num_buyers)]
else:
  # Use the predefined list of names if sufficient
  print(f'Info: Using {num_agents} predefined names for agents.')
  random.shuffle(names)
  seller_names = names[:num_sellers]
  buyer_names = names[num_sellers:num_agents]

  seller_profiles = [{'name': name} for name in seller_names]
  buyer_profiles = [{'name': name} for name in buyer_names]

# Add randomized efficiencies to sellers
for seller in seller_profiles:
  seller['efficiency'] = round(random.uniform(0.8, 1.3), 2)

# Generate 50 buyer profiles with cash distributed using a Pareto (power-law) distribution.
min_cash = 50.0  # The scale parameter (x_m) of the distribution.
# The shape parameter (alpha). Values > 1 are typical. Smaller values create a more extreme tail (more inequality).
# A value of ~1.16 represents the 80/20 rule (Pareto principle). We use 1.5 for a strong, but not extreme, skew.
distribution_shape = 1.5
# An effective maximum to prevent unrealistic outlier values in the simulation.
max_cash_cap = 25000.0

# Generate values from the Pareto distribution. The formula (np.random.pareto(a) + 1) * xm
# generates values with a minimum of xm. We generate all 50 values at once.
cash_values = np.random.pareto(distribution_shape, num_buyers) * min_cash
# Clip the values to our defined maximum cap to prevent outliers from breaking the simulation.
cash_values = np.clip(cash_values, min_cash, max_cash_cap)

# Add cash to buyers
for i, buyer in enumerate(buyer_profiles):
  buyer['cash'] = f'{round(cash_values[i], 2)}'

# --- Select ONE CATEGORY for all producers ---
# This focuses the market simulation on direct competition within a single category.
selected_category = random.choice(list(DETAILED_GOODS.keys()))
print(f'--- Market category selected for this run: {selected_category} ---')

# Get the list of producible goods within the selected category
producible_goods_in_category = [
    (selected_category, qual, item_name, details)
    for qual, items in DETAILED_GOODS[selected_category].items()
    for item_name, details in items.items()
]
producible_goods_in_category = producible_goods_in_category[:num_items]

# --- Programmatically process agents to assign goods and calculate costs ---
agent_data = []

# Assign goods and costs to the 50 sellers
for seller in seller_profiles:
  category, quality, item_name, good_details = random.choice(
      producible_goods_in_category
  )

  seller_data = seller.copy()
  seller_data['type'] = 'producer'
  seller_data['good_to_sell'] = item_name
  # Cost = base_cost of the item * seller's personal efficiency modifier
  seller_data['production_cost'] = round(
      good_details['price'] * seller['efficiency'], 2
  )
  seller_data['inventory'] = (
      10 * num_rounds
  )  # Increased inventory for a longer simulation
  agent_data.append(seller_data)

# Add the 50 buyers to the final list
for buyer in buyer_profiles:
  buyer_data = buyer.copy()
  buyer_data['type'] = 'consumer'
  random_clothing = random.choice(
      list(DETAILED_GOODS['Clothing']['Low'].keys())
  )
  buyer_data['inventory'] = {random_clothing: 1}
  agent_data.append(buyer_data)

# --- Setup Simulation Components with the new data ---
names = [agent['name'] for agent in agent_data]
goods = get_all_goods_from_spec(DETAILED_GOODS)
agents = make_agents_from_data(agent_data)

component_kwargs = {
    'components': [
        actor_components.observation.DEFAULT_OBSERVATION_COMPONENT_KEY
    ],
    'agents': agents,
    'goods': goods,
}

# --- Programmatically Generate Agent Instances ---
instances = []
for agent in agent_data:
  if agent['type'] == 'producer':
    goal_text = (
        f"You are a seller of {agent['good_to_sell']}. Your cost to produce"
        f" each unit is ${agent['production_cost']:.2f}. Your goal is to sell"
        ' your stock for a profit. You must sell for more than your cost to be'
        ' profitable.'
    )
    prefab = 'basic__Entity'
  else:  # Buyer
    goal_text = (
        f'You are a buyer at the marketplace. Your goal is to purchase goods'
        f' that match your preference for items of a certain quality and'
        f' category given your budget. You will try to buy them for a good'
        f' value price.'
    )
    prefab = 'basic__Entity'

  instance_config = prefab_lib.InstanceConfig(
      prefab=prefab,
      role=prefab_lib.Role.ENTITY,
      params={'name': agent['name'], 'goal': goal_text},
  )
  instances.append(instance_config)

# --- Add Game Master and Initializer Instances ---

instances.append(
    prefab_lib.InstanceConfig(
        prefab='marketplace__GameMaster',
        role=prefab_lib.Role.GAME_MASTER,
        params={
            'name': 'MarketplaceGM',
            'experiment_component_class': MarketPlace,
            'experiment_component_init_kwargs': component_kwargs,
        },
    )
)

# Generic shared memories for the larger group
all_player_names = ', '.join(names)
instances.append(
    prefab_lib.InstanceConfig(
        prefab='formative_memories_initializer__GameMaster',
        role=prefab_lib.Role.INITIALIZER,
        params={
            'name': 'initial setup rules',
            'next_game_master_name': 'MarketplaceGM',
            'shared_memories': [
                (
                    'There is a marketplace in Los Angeles where all'
                    f' residents, including {all_player_names}, come to trade.'
                ),
                (
                    'Some residents are producers of various goods, while'
                    ' others are buyers.'
                ),
            ],
        },
    )
)

# --- Final Configuration and Simulation Initialization ---

default_premise = """You are in a marketplace that buys and sells goods for food, clothing, and gadgets of low, mid, and high quality. Interact with other participants to achieve your goals.
"""

config = prefab_lib.Config(
    default_premise=default_premise,
    default_max_steps=num_rounds,
    prefabs=prefabs,
    instances=instances,
)

engine = simultaneous.Simultaneous()

#  Initialize the simulation
realistic_sim = simulation.Simulation(
    config=config, model=model, embedder=embedder, engine=engine
)

In [ ]:
# @title Run the simulation

results_log = realistic_sim.play()

In [ ]:
# @title Display the log
display.HTML(results_log)

# Plot Results

In [ ]:
# @title Plotting Functions


def find_equilibrium(prices, supply, demand):
  """Finds the equilibrium price and quantity from discrete supply and demand curves.

  This function identifies the intersection point, which represents the
  theoretical
  market-clearing price and quantity.

  Args:
      prices (list): A sorted list of unique prices from the order book.
      supply (list): The cumulative supply quantity at each price.
      demand (list): The cumulative demand quantity at each price.

  Returns:
      tuple: A tuple of (equilibrium_price, equilibrium_quantity). Returns
             (None, 0) if no trade is possible (i.e., the lowest ask price
             is higher than the highest bid price).
  """
  # Case 1: No overlap between supply and demand curves.
  # This happens if the highest price a buyer is willing to pay is less than
  # the lowest price a seller is willing to accept. No trades can occur.
  # We detect this by checking if the maximum demand is zero at the lowest price
  # where supply starts to appear.
  if not any(d > 0 for d in demand) or not any(s > 0 for s in supply):
    return None, 0  # No demand or no supply at all.

  first_supply_idx = next((i for i, s in enumerate(supply) if s > 0), None)
  if first_supply_idx is not None and demand[first_supply_idx] == 0:
    return None, 0  # No demand at the minimum supply price.

  # Case 2: Find the intersection point.
  # We iterate through the prices to find the last point where demand is
  # still greater than or equal to supply. This point represents the
  # clearing price and quantity for all possible trades.
  eq_price = None
  eq_quantity = 0

  for i in range(len(prices)):
    if demand[i] >= supply[i]:
      # The potential quantity that could be traded is the minimum
      # of the supply and demand at this price point.
      # However, since we are looking for the final clearing price,
      # we consider the supply available at this level.
      eq_quantity = supply[i]
      eq_price = prices[i]
    else:
      # The first time demand drops below supply, we've found our
      # crossover region. The previous point was the equilibrium.
      break

  return eq_price, eq_quantity


def plot_supply_demand_history(
    curve_history, empirical_price_history=None, good_id='Food_Low'
):
  """Plots the supply and demand curves for each round in the history,

  and optionally overlays the actual transaction prices.

  Args:
      curve_history (dict): History of supply/demand curves {round: {good:
        {data}}}.
      empirical_price_history (list, optional): A list of dicts with actual
        transaction prices for each round. e.g., [{'Food_Low': 2.0}, ...].
      good_id (str): The specific good to plot.
  """
  num_rounds = len(curve_history)
  if num_rounds == 0:
    print('Curve history is empty. Nothing to plot.')
    return

  # Create subplots for each round.
  cols = 3
  rows = int(np.ceil(num_rounds / cols))
  fig, axes = plt.subplots(
      rows, cols, figsize=(cols * 5.5, rows * 5), squeeze=False
  )
  axes = axes.flatten()

  for i, (round_idx, round_data) in enumerate(curve_history.items()):
    ax = axes[i]

    market_data = round_data.get(good_id, {})
    prices = market_data.get('prices', [])
    supply = market_data.get('supply', [])
    demand = market_data.get('demand', [])

    # Plot supply and demand curves
    if prices:
      ax.step(
          supply,
          prices,
          where='post',
          label='Supply',
          color='cornflowerblue',
          linewidth=2,
      )
      ax.step(
          demand,
          prices,
          where='post',
          label='Demand',
          color='salmon',
          linewidth=2,
      )

    # Find and plot the theoretical equilibrium point
    eq_price, eq_quantity = find_equilibrium(prices, supply, demand)

    title_str = f'Round {round_idx}\n'
    if eq_price is not None:
      ax.plot(
          eq_quantity, eq_price, 'ko', markersize=8, label=f'Theoretical Eq.'
      )
      ax.hlines(
          y=eq_price,
          xmin=0,
          xmax=eq_quantity,
          color='grey',
          linestyle='--',
          linewidth=1.5,
      )
      ax.vlines(
          x=eq_quantity,
          ymin=0,
          ymax=eq_price,
          color='grey',
          linestyle='--',
          linewidth=1.5,
      )
      title_str += f'Theoretical Eq: ${eq_price:.2f}, Qty: {eq_quantity}'
    else:
      title_str += 'No Trade Equilibrium'

    # Plot the empirical price if available
    if empirical_price_history and i < len(empirical_price_history):
      emp_price = empirical_price_history[i].get(good_id)
      if emp_price is not None and not np.isnan(emp_price):
        ax.axhline(
            y=emp_price,
            color='darkviolet',
            linestyle=':',
            linewidth=2.5,
            label=f'Actual Price (${emp_price:.2f})',
        )

    ax.set_title(title_str)
    ax.set_xlabel('Quantity')
    ax.set_ylabel('Price ($)')
    ax.grid(True, linestyle=':', alpha=0.6)

    # Set reasonable limits
    max_qty = max(max(supply, default=0), max(demand, default=0))
    ax.set_xlim(left=-0.5, right=max_qty + 1 if max_qty > 0 else 10)
    ax.set_ylim(bottom=0)
    ax.legend()

  # Hide any unused subplots
  for j in range(num_rounds, len(axes)):
    fig.delaxes(axes[j])

  fig.suptitle(f'Supply and Demand Curves for "{good_id}"', fontsize=16, y=1.03)
  plt.tight_layout()
  plt.show()


def plot_bid_ask_spread(df, product_to_plot):
  """Processes market data for a specific product and plots its

  bid-ask spread over time.

  Args:
      df (pd.DataFrame): The input DataFrame containing market data. Must
        include 'good', 'round', 'day', 'price', 'demand', and 'supply' columns.
      product_to_plot (str): The name of the product to plot.
  """

  # --- Process the Data from the curve_history Object ---
  processed_data = []
  rounds = df['round'].unique()

  curve_history = df[df['good'] == product_to_plot]

  for round_num in rounds:
    market_data = curve_history[curve_history['round'] == round_num]
    if product_to_plot not in market_data['good'].values:
      continue  # Skip if product doesn't exist in this round

    prices = market_data['price'].values
    demands = market_data['demand'].values
    supplies = market_data['supply'].values

    # Find the best bid (highest price with non-zero demand)
    bid_prices = [price for price, demand in zip(prices, demands) if demand > 0]
    best_bid = max(bid_prices) if bid_prices else np.nan

    # Find the best ask (lowest price with non-zero supply)
    ask_prices = [
        price for price, supply in zip(prices, supplies) if supply > 0
    ]
    best_ask = min(ask_prices) if ask_prices else np.nan

    # Calculate the spread
    spread = (
        best_ask - best_bid
        if not np.isnan(best_bid) and not np.isnan(best_ask)
        else np.nan
    )

    processed_data.append({
        'Round': round_num,
        'Best Bid': best_bid,
        'Best Ask': best_ask,
        'Spread': spread,
    })

  # Create a DataFrame from the processed data
  market_df = pd.DataFrame(processed_data)

  # --- 3. Plot the Data ---
  if not market_df.empty:
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(14, 8))

    # Plot the best bid and best ask lines
    # Using .dropna() to avoid plotting gaps with straight lines
    ax.plot(
        market_df['Round'],
        market_df['Best Bid'],
        marker='o',
        linestyle='-',
        color='royalblue',
        label='Best Bid',
    )
    ax.plot(
        market_df['Round'],
        market_df['Best Ask'],
        marker='o',
        linestyle='-',
        color='coral',
        label='Best Ask',
    )

    # Shade the area between the bid and ask to represent the spread
    ax.fill_between(
        market_df['Round'],
        market_df['Best Bid'],
        market_df['Best Ask'],
        color='skyblue',
        alpha=0.4,
        label='Bid-Ask Spread',
    )

    # --- 4. Customize the Plot ---
    ax.set_title(
        f'Bid-Ask Spread by Round for\n{product_to_plot}',
        fontsize=16,
        fontweight='bold',
    )
    ax.set_xlabel('Round', fontsize=12)
    ax.set_ylabel('Price', fontsize=12)
    ax.set_xticks(market_df['Round'])
    ax.legend(fontsize=10)
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)

    plt.tight_layout()
    plt.show()
  else:
    print(
        f"\nCould not find data for '{product_to_plot}' in the history object."
        ' Please check the name.'
    )

## Number of Purchases per Good

In [ ]:
trade_history = (
    realistic_sim.game_masters[1]
    .get_component(
        gm_components.make_observation.DEFAULT_MAKE_OBSERVATION_COMPONENT_KEY
    )
    .trade_history
)
df_trades = pd.DataFrame(trade_history)

# Generate a complete list of all goods
all_goods_formatted = []
for category, category_data in DETAILED_GOODS.items():
  for quality, quality_data in category_data.items():
    for good_name in quality_data.keys():
      all_goods_formatted.append(good_name)

# Calculate purchase counts
successful_bids = df_trades[
    (df_trades['transaction_occurred']) & (df_trades['side'] == 'bid')
]
purchase_counts = successful_bids['good'].value_counts()

# Reindex to include all goods, filling missing ones with 0
purchase_counts_all = purchase_counts.reindex(
    all_goods_formatted, fill_value=0
).sort_values(ascending=False)

# --- Plot the Horizontal Bar Chart ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.figure(figsize=(12, 10))  # Adjust figure size for a horizontal layout
sns.barplot(
    x=purchase_counts_all.values,
    y=purchase_counts_all.index,
    palette='viridis',
    orient='h',
)
plt.xlabel('Number of Purchases', fontsize=12)
plt.ylabel('Good', fontsize=12)
plt.title('Total Number of Purchases per Good', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## Supply Demand Curves

In [ ]:
# The data for the supply and demand curves
curve_history = (
    realistic_sim.game_masters[1]
    .get_component(
        gm_components.make_observation.DEFAULT_MAKE_OBSERVATION_COMPONENT_KEY
    )
    .curve_history
)

# The data for the actual transaction prices from the simulation
price_history = (
    realistic_sim.game_masters[1]
    .get_component(
        gm_components.make_observation.DEFAULT_MAKE_OBSERVATION_COMPONENT_KEY
    )
    .history
)

# Choose a valid ID from your curve_history data
good_info = producible_goods_in_category[0]
good_to_plot = good_info[2]

# Call the function with your data and the chosen ID
plot_supply_demand_history(
    curve_history, empirical_price_history=price_history, good_id=good_to_plot
)

## Bid Ask Spreads

In [ ]:
# Get a list of all unique products in the DataFrame
sd_data = []
for round_num, goods_data in curve_history.items():
  for good_name, curve_data in goods_data.items():
    prices = curve_data['prices']
    supplies = curve_data['supply']
    demands = curve_data['demand']
    for i in range(len(prices)):
      record = {
          'round': round_num,
          'good': good_name,
          'price_point': i,
          'price': prices[i],
          'supply': supplies[i],
          'demand': demands[i],
      }
      sd_data.append(record)

# Convert list of records to DataFrame
df_sd = pd.DataFrame(sd_data)
all_products = df_sd['good'].unique()

# Loop through each product and generate a plot
for product in all_products:
  plot_bid_ask_spread(df_sd, product)

```
Copyright 2025 DeepMind Technologies Limited.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```